# Santander Bootcamp Ciência de Dados com Python 2023 (Explorando IA Generativa em um Pipeline de ETL com Python)



**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SBCDP2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [ ]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sbcdp2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

_Entendendo o arquivo:_ **SBCDP2023 - Santander Bootcamp Ciência de Dados com Python 2023**

- [X] Extrair os IDS do arquivo CSV

In [ ]:
import pandas as pd

df = pd.read_csv('SBCDP2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[597, 598, 599]


- [x] Obter os dados de cada ID usando a API da Santander Dev Week 2023

In [ ]:
import requests
import json

# Função para obter detalhes de um usuário por meio do arquivo CSV com uma solicitação GET à API da Santander Dev Week 2023
def get_user(id):
  response = requests.get(f'{sbcdp2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

# Lista de usuários válidos obtidos através da função get_user
users = [user for id in user_ids if (user := get_user(id)) is not None]

# Imprimir os detalhes dos usuários em formato JSON formatado
for user in users:
    user['news'] = [] # Limpar a lista de 'news' existente pois o exercício já foi executado anteriormente
print(json.dumps(users, indent=3))

[
   {
      "id": 597,
      "name": "James",
      "account": {
         "id": 607,
         "number": "000002-02",
         "agency": "0001",
         "balance": 0.0,
         "limit": 5000.0
      },
      "card": {
         "id": 571,
         "number": "**** **** **** 0220",
         "limit": 1000.0
      },
      "features": [],
      "news": []
   },
   {
      "id": 598,
      "name": "Curry",
      "account": {
         "id": 608,
         "number": "000003-03",
         "agency": "0001",
         "balance": 0.0,
         "limit": 5000.0
      },
      "card": {
         "id": 572,
         "number": "**** **** **** 0330",
         "limit": 1000.0
      },
      "features": [],
      "news": []
   },
   {
      "id": 599,
      "name": "Jordan",
      "account": {
         "id": 609,
         "number": "000004-04",
         "agency": "0001",
         "balance": 0.0,
         "limit": 5000.0
      },
      "card": {
         "id": 573,
         "number": "**** **** **** 0440",

## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
!pip install openai

In [ ]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = 'TODO'

- [X] Devido à limitação de não ter acesso à créditos da API da OpenAI, a solução foi criar manualmente com as mensagens personalizadas em uma função diferente da proposta

In [ ]:
# Função para gerar a mensagem personalizada
def send_message(user):
    messages= f"Jogue bem dentro e fora da quadra: Invista no seu amanhã, {user['name']}! #SucessoForaDaQuadra #FuturoBrilhante #InvestindoNoAmanhã"
    return messages

# Loop para atualizar os usuários
for user in users:
    user['news'] = [] # Limpar a lista de 'news' existente
    news = send_message(user) # Gera a mensagem personalizada usando a função send_message
    print(news) # Imprime a mensagem gerada
    user['news'].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news # Adiciona a mensagem à lista de 'news' do usuário
    })

Jogue bem dentro e fora da quadra: Invista no seu amanhã, James! #SucessoForaDaQuadra #FuturoBrilhante #InvestindoNoAmanhã
Jogue bem dentro e fora da quadra: Invista no seu amanhã, Curry! #SucessoForaDaQuadra #FuturoBrilhante #InvestindoNoAmanhã
Jogue bem dentro e fora da quadra: Invista no seu amanhã, Jordan! #SucessoForaDaQuadra #FuturoBrilhante #InvestindoNoAmanhã


## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [ ]:
# Função para atualizar os detalhes do usuário na API
def update_user(user):
    response = requests.put(f"{sbcdp2023_api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

# Loop para iterar pelos usuários e atualizar suas informações
for user in users:
    success = update_user(user) # Chama a função para atualizar o usuário na API
    print(f"User {user['name']} updated? {success}!") # Imprime se a atualização foi bem-sucedida ou não

User James updated? True!
User Curry updated? True!
User Jordan updated? True!
